<a href="https://colab.research.google.com/github/profteachkids/CHE4071_Fall2023/blob/main/LevelConcTemp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import jax.numpy as jnp
import jax
jax.config.update('jax_enable_x64',True)
from plotly.subplots import make_subplots
from scipy.optimize import root
from scipy.integrate import solve_ivp

In [52]:
s1T=90.  #C
s2T=50.
s3T=70.

s1C=2. #mol/L
s2C=1.
s3C=0.

rho=1 #kg/L
Cp=4190 #J/kg

T0 = 75. #C
C0 = 1.2 #mol/L
h0 = 1.  #m

A = 1.  #m2
a = 0.05  #m2
tend = 50.

In [53]:
sq2g = (2*9.81)**0.5
def rhs(t, hCT, qs):
    h,C,T = hCT
    q1, q2, q3 = qs
    qout = sq2g*(h**0.5)*a
    dh=(q1+q2+q3 - qout)/A
    dC = (q1*s1C + q2*s2C + q3*s3C - qout*C - C*A*dh)/(h*A)          # dCV = V dC + C dV = Ah dC + CA dh = q1*c1 + q2*c2 + q3*c3 - qout*C
    # d(rho Cp V T) = rho*Cp*(q1*s1T + q2*s2T + q3*s3T) - rho*Cp*qout*T
    # V dT + T dV = (q1*s1T + q2*s2T + q3*s3T) - qout*T
    dT = ((q1*s1T + q2*s2T + q3*s3T) - qout*T - T*A*dh)/(h*A)
    return jnp.array([dh, dC, dT])

In [54]:
qi_ss=root(lambda qs: rhs(0., jnp.array([h0, C0, T0]), qs), jnp.ones(3))

In [55]:
qs = qi_ss.x * jnp.array([0.9, 1.1, 1.2])

In [56]:
res=solve_ivp(rhs, (0,tend), jnp.array([h0, C0, T0]), args=(qs,), dense_output=True, method='LSODA', jac=jax.jacobian(rhs,1))

In [57]:
tplot=jnp.linspace(0,tend,250)
h,C,T = res.sol(tplot)
fig=make_subplots(rows=1,cols=3)
fig.add_scatter(x=tplot,y=h,mode='lines',row=1,col=1,name='h')
fig.add_scatter(x=tplot,y=C,mode='lines',row=1,col=2,name='C')
fig.add_scatter(x=tplot,y=T,mode='lines',row=1,col=3,name='T')
fig.update_layout(width=1200, height=400, template='plotly_dark')